In [1]:
# img = {
#     'content': {
#         'photo_type': {
#             'label': 'FRONTAL'
#         },
#         'vehicle_type': {
#             'label':  'Automóvil'
#         }
#     }
# }
img = {
    'content': {
        'photo_type': {
            'label': 'TRASERA'
        },
        'vehicle_type': {
            'label':  'Motocicleta'
        }
    }
}
if img['content']['photo_type']['label'] in ['TRASERA']:
    print('TRASERA', img)
elif img['content']['photo_type']['label'] in ['FRONTAL']:
    if img['content']['vehicle_type']['label'] in ['Automóvil', 'Camión']: 
        print('FRONTAL', img)

TRASERA {'content': {'photo_type': {'label': 'TRASERA'}, 'vehicle_type': {'label': 'Motocicleta'}}}


In [2]:
# content = {"filename": "SDBYH5_RFVIN.jpg", "status": "REPEAT", "message": None, "applied_fuctions": ["classify_vehicle_photo", "extract_text:vin"], "n_labels": 1, "vehicle_type": {"label": None, "probability": None}, "photo_type": {"label": "VIN", "probability": 0.9993376}, "vin": {"text": None, "probability": None, "lines": [[":980578975", 0.647], ["GB023037", 0.71]]}, "plate": {"text": None, "probability": None}}
content = { "applied_fuctions": [ "classify_vehicle_photo", "extract_text:vin" ], "n_info": 1, "photo_type": { "label": "VIN", "probability": 0.99991226 }, "plate": { "probability": None, "text": None }, "status": "REPEAT", "vehicle_type": { "label": None, "probability": None }, "vin": { "lines": [ [ "BENELLI Q.J.SRL", 0.976 ], [ "L3e-A2", 0.993 ], [ "e13*168/2013*00123", 0.985 ], [ "* ZBNP160071 BA913192", 0.742 ], [ "91dB(A)-4250min-1", 0.95 ], [ "35.0 kW max 427kg", 0.989 ] ], "probability": None, "text": None } }
' '.join(word for word, prob in content['vin']['lines'])

'BENELLI Q.J.SRL L3e-A2 e13*168/2013*00123 * ZBNP160071 BA913192 91dB(A)-4250min-1 35.0 kW max 427kg'

In [3]:
import base64
from PIL import Image
from io import BytesIO

def image_to_base64(image_path):
    # Open the image file
    with Image.open(image_path, encoding='utf-8') as image:
        # Create a BytesIO object
        buffered = BytesIO()
        # Save the image to the BytesIO object in PNG format
        image.save(buffered, format='PNG')
        # Get the byte data from the BytesIO object
        img_byte = buffered.getvalue()
        # Encode the byte data to base64
        img_base64 = base64.b64encode(img_byte)
        # Convert base64 bytes to string
        img_base64_string = img_base64.decode('utf-8')
        return img_base64_string

# Example usage
image_path = IMG_PATH
base64_string = image_to_base64(image_path)
# print(base64_string)
print(len(base64_string))

NameError: name 'IMG_PATH' is not defined

In [ ]:
from vision_toolkit import VehicleImage
from pprint import pprint

IMG_PATH = 'work-zone/Fase2/NEGATIVO/Tests/TEST FIAT - VIN CALIDAD ALTA.jpeg' # VIN
# IMG_PATH = 'work-zone/Fase2/DOCUMENTO/VXQ671_RFRTV.jpg'                 # DOCUMENTO: Plate/VIN
# IMG_PATH = 'work-zone/Fase2/Automóvil/FRONTAL/1BLSR5_RFFRE.jpg'         # PLATE
# IMG_PATH = 'work-zone/Fase2/Camión/INTERIOR/WZD418 - INTERIOR.jpeg'     # INTERIOR Camión
# IMG_PATH = 'work-zone/Fase2/Motocicleta/FRONTAL/WLF34E - FRONTAL.jpeg'  # FRONTAL Motocicleta

base64_string = image_to_base64(image_path)

with open(IMG_PATH, mode='rb') as img_binary:
    input_request = {
        'files': [img_binary]
    }

    # Load image
    img = VehicleImage(input_request['files'][0])
    img.show()

    # Logic
    img.classify_vehicle_photo()                                        # [NEXT] extract_text, option cropped

    # No classification (likely due to image quality) -> REPEAT
    if (img.content['n_info'] == 0):
        img.content['status'] = 'REPEAT'

    # Classification 1 (VIN, DOCUMENTO): Get VIN or Plate -> ???
    if img.content['n_info'] == 1:
        if img.content['photo_type']['label'] == 'VIN':
            img.extract_text(content_type='vin')                            # [NEXT] clean text / URL
        if img.content['photo_type']['label'] == 'DOCUMENTO':
            img.extract_entities_from_doc()                                 # [NEXT] use confidence

        # # Classification 2 (FRONTAL R2, TRASERA R2): Get plata -> ???
        # if img.content['n_info'] == 2:
        #     if (img.content['photo_type']['label'] in ['FRONTAL', 'TRASERA']):  # [NEXT] MOTO
        #         cropped_image = img.detect_plate()                              # [NEXT] don't save cropped img / URL
        #     else:
        #         img.content['status'] = 'SUCCESS'
        # # Classification 3+: Model error -> REVIEW
        # if (img.content['n_info'] >= 3):
        #     img.content['status'] = 'REVIEW'

    pprint(img.content, width=200, sort_dicts=False)


In [ ]:
import re
patterns = {
    'vin'  : r'\b(?=.*[0-9])(?=.*[A-Z])[0-9A-Z]{17}\b',
    'plate': r'^[A-Z]{3}\d{3}$',
}

In [ ]:
# lines = [{'text': '9BD363B32PYZ78594', 'boundingPolygon': [{'x': 466, 'y': 744}, {'x': 728, 'y': 744}, {'x': 728, 'y': 767}, {'x': 466, 'y': 766}], 'words': [{'text': '9BD363B32PYZ78594', 'boundingPolygon': [{'x': 467, 'y': 745}, {'x': 723, 'y': 745}, {'x': 723, 'y': 766}, {'x': 467, 'y': 765}], 'confidence': 0.99}]}, {'text': 'FIAT PULSE IMPETUS AT', 'boundingPolygon': [{'x': 440, 'y': 768}, {'x': 751, 'y': 768}, {'x': 751, 'y': 792}, {'x': 440, 'y': 791}], 'words': [{'text': 'FIAT', 'boundingPolygon': [{'x': 442, 'y': 769}, {'x': 497, 'y': 769}, {'x': 497, 'y': 792}, {'x': 442, 'y': 791}], 'confidence': 0.987}, {'text': 'PULSE', 'boundingPolygon': [{'x': 504, 'y': 769}, {'x': 583, 'y': 770}, {'x': 583, 'y': 793}, {'x': 504, 'y': 792}], 'confidence': 0.997}, {'text': 'IMPETUS', 'boundingPolygon': [{'x': 595, 'y': 770}, {'x': 705, 'y': 770}, {'x': 705, 'y': 793}, {'x': 595, 'y': 793}], 'confidence': 0.994}, {'text': 'AT', 'boundingPolygon': [{'x': 718, 'y': 769}, {'x': 749, 'y': 769}, {'x': 749, 'y': 793}, {'x': 718, 'y': 793}], 'confidence': 0.998}]}, {'text': 'ROJO MONTE CARLO NEGRO', 'boundingPolygon': [{'x': 411, 'y': 793}, {'x': 778, 'y': 794}, {'x': 778, 'y': 817}, {'x': 411, 'y': 816}], 'words': [{'text': 'ROJO', 'boundingPolygon': [{'x': 413, 'y': 794}, {'x': 476, 'y': 794}, {'x': 475, 'y': 817}, {'x': 412, 'y': 817}], 'confidence': 0.989}, {'text': 'MONTE', 'boundingPolygon': [{'x': 489, 'y': 794}, {'x': 577, 'y': 795}, {'x': 577, 'y': 817}, {'x': 489, 'y': 817}], 'confidence': 0.994}, {'text': 'CARLO', 'boundingPolygon': [{'x': 589, 'y': 795}, {'x': 670, 'y': 795}, {'x': 670, 'y': 817}, {'x': 589, 'y': 817}], 'confidence': 0.993}, {'text': 'NEGRO', 'boundingPolygon': [{'x': 685, 'y': 795}, {'x': 771, 'y': 794}, {'x': 772, 'y': 817}, {'x': 685, 'y': 817}], 'confidence': 0.993}]}]
# lines = [{'text': 'NISSAN', 'boundingPolygon': [{'x': 361, 'y': 427}, {'x': 578, 'y': 428}, {'x': 578, 'y': 476}, {'x': 360, 'y': 475}], 'words': [{'text': 'NISSAN', 'boundingPolygon': [{'x': 363, 'y': 427}, {'x': 571, 'y': 429}, {'x': 570, 'y': 477}, {'x': 361, 'y': 476}], 'confidence': 0.994}]}, {'text': 'TYPE', 'boundingPolygon': [{'x': 302, 'y': 479}, {'x': 335, 'y': 479}, {'x': 335, 'y': 498}, {'x': 302, 'y': 499}], 'words': [{'text': 'TYPE', 'boundingPolygon': [{'x': 302, 'y': 479}, {'x': 334, 'y': 479}, {'x': 334, 'y': 499}, {'x': 302, 'y': 499}], 'confidence': 0.979}]}, {'text': 'TIPO', 'boundingPolygon': [{'x': 300, 'y': 501}, {'x': 332, 'y': 500}, {'x': 333, 'y': 520}, {'x': 301, 'y': 520}], 'words': [{'text': 'TIPO', 'boundingPolygon': [{'x': 301, 'y': 500}, {'x': 331, 'y': 500}, {'x': 332, 'y': 520}, {'x': 301, 'y': 520}], 'confidence': 0.967}]}, {'text': 'NO. DE CHASSIS', 'boundingPolygon': [{'x': 98, 'y': 552}, {'x': 262, 'y': 545}, {'x': 263, 'y': 566}, {'x': 99, 'y': 574}], 'words': [{'text': 'NO.', 'boundingPolygon': [{'x': 98, 'y': 553}, {'x': 136, 'y': 551}, {'x': 137, 'y': 572}, {'x': 99, 'y': 574}], 'confidence': 0.996}, {'text': 'DE', 'boundingPolygon': [{'x': 141, 'y': 551}, {'x': 167, 'y': 550}, {'x': 167, 'y': 571}, {'x': 141, 'y': 572}], 'confidence': 0.989}, {'text': 'CHASSIS', 'boundingPolygon': [{'x': 177, 'y': 549}, {'x': 260, 'y': 546}, {'x': 260, 'y': 567}, {'x': 177, 'y': 570}], 'confidence': 0.993}]}, {'text': '3N6CD33B7HK802000', 'boundingPolygon': [{'x': 316, 'y': 522}, {'x': 627, 'y': 511}, {'x': 628, 'y': 549}, {'x': 317, 'y': 561}], 'words': [{'text': '3N6CD33B7HK802000', 'boundingPolygon': [{'x': 319, 'y': 523}, {'x': 626, 'y': 511}, {'x': 626, 'y': 550}, {'x': 320, 'y': 562}], 'confidence': 0.993}]}, {'text': 'D23', 'boundingPolygon': [{'x': 476, 'y': 477}, {'x': 528, 'y': 477}, {'x': 528, 'y': 515}, {'x': 476, 'y': 515}], 'words': [{'text': 'D23', 'boundingPolygon': [{'x': 476, 'y': 477}, {'x': 527, 'y': 477}, {'x': 527, 'y': 515}, {'x': 476, 'y': 515}], 'confidence': 0.998}]}, {'text': 'CHASSIS NO.', 'boundingPolygon': [{'x': 99, 'y': 528}, {'x': 230, 'y': 525}, {'x': 231, 'y': 545}, {'x': 99, 'y': 550}], 'words': [{'text': 'CHASSIS', 'boundingPolygon': [{'x': 99, 'y': 529}, {'x': 186, 'y': 527}, {'x': 186, 'y': 547}, {'x': 99, 'y': 550}], 'confidence': 0.994}, {'text': 'NO.', 'boundingPolygon': [{'x': 195, 'y': 527}, {'x': 230, 'y': 526}, {'x': 230, 'y': 546}, {'x': 195, 'y': 547}], 'confidence': 0.994}]}, {'text': 'MODEL', 'boundingPolygon': [{'x': 98, 'y': 574}, {'x': 170, 'y': 573}, {'x': 170, 'y': 593}, {'x': 99, 'y': 596}], 'words': [{'text': 'MODEL', 'boundingPolygon': [{'x': 98, 'y': 575}, {'x': 167, 'y': 573}, {'x': 168, 'y': 593}, {'x': 99, 'y': 597}], 'confidence': 0.996}]}, {'text': 'CVL4LWYD23IYP-', 'boundingPolygon': [{'x': 293, 'y': 563}, {'x': 521, 'y': 552}, {'x': 523, 'y': 591}, {'x': 295, 'y': 603}], 'words': [{'text': 'CVL4LWYD23IYP-', 'boundingPolygon': [{'x': 294, 'y': 564}, {'x': 519, 'y': 553}, {'x': 522, 'y': 590}, {'x': 295, 'y': 604}], 'confidence': 0.879}]}, {'text': 'MODELO', 'boundingPolygon': [{'x': 98, 'y': 598}, {'x': 182, 'y': 593}, {'x': 184, 'y': 612}, {'x': 98, 'y': 618}], 'words': [{'text': 'MODELO', 'boundingPolygon': [{'x': 98, 'y': 599}, {'x': 180, 'y': 594}, {'x': 180, 'y': 613}, {'x': 98, 'y': 619}], 'confidence': 0.996}]}, {'text': 'COLOR,TRIM', 'boundingPolygon': [{'x': 97, 'y': 620}, {'x': 224, 'y': 612}, {'x': 225, 'y': 632}, {'x': 98, 'y': 641}], 'words': [{'text': 'COLOR,TRIM', 'boundingPolygon': [{'x': 98, 'y': 621}, {'x': 213, 'y': 613}, {'x': 214, 'y': 632}, {'x': 98, 'y': 641}], 'confidence': 0.962}]}, {'text': 'KH3 G', 'boundingPolygon': [{'x': 461, 'y': 592}, {'x': 535, 'y': 588}, {'x': 537, 'y': 622}, {'x': 462, 'y': 629}], 'words': [{'text': 'KH3', 'boundingPolygon': [{'x': 461, 'y': 592}, {'x': 514, 'y': 588}, {'x': 517, 'y': 624}, {'x': 462, 'y': 628}], 'confidence': 0.993}, {'text': 'G', 'boundingPolygon': [{'x': 521, 'y': 588}, {'x': 533, 'y': 588}, {'x': 536, 'y': 623}, {'x': 524, 'y': 623}], 'confidence': 0.993}]}, {'text': 'COLOR, GUARNICION', 'boundingPolygon': [{'x': 97, 'y': 642}, {'x': 286, 'y': 628}, {'x': 287, 'y': 648}, {'x': 98, 'y': 663}], 'words': [{'text': 'COLOR,', 'boundingPolygon': [{'x': 98, 'y': 642}, {'x': 167, 'y': 637}, {'x': 168, 'y': 658}, {'x': 98, 'y': 663}], 'confidence': 0.993}, {'text': 'GUARNICION', 'boundingPolygon': [{'x': 171, 'y': 637}, {'x': 285, 'y': 628}, {'x': 286, 'y': 648}, {'x': 172, 'y': 658}], 'confidence': 0.993}]}, {'text': 'ENGINE', 'boundingPolygon': [{'x': 98, 'y': 664}, {'x': 171, 'y': 659}, {'x': 172, 'y': 678}, {'x': 99, 'y': 683}], 'words': [{'text': 'ENGINE', 'boundingPolygon': [{'x': 98, 'y': 664}, {'x': 170, 'y': 660}, {'x': 170, 'y': 679}, {'x': 99, 'y': 684}], 'confidence': 0.994}]}, {'text': 'YD25DDTI 2488 cc', 'boundingPolygon': [{'x': 429, 'y': 633}, {'x': 656, 'y': 612}, {'x': 659, 'y': 647}, {'x': 432, 'y': 668}], 'words': [{'text': 'YD25DDTI', 'boundingPolygon': [{'x': 430, 'y': 634}, {'x': 559, 'y': 622}, {'x': 562, 'y': 658}, {'x': 433, 'y': 668}], 'confidence': 0.943}, {'text': '2488', 'boundingPolygon': [{'x': 566, 'y': 622}, {'x': 616, 'y': 617}, {'x': 619, 'y': 652}, {'x': 569, 'y': 657}], 'confidence': 0.993}, {'text': 'cc', 'boundingPolygon': [{'x': 623, 'y': 616}, {'x': 654, 'y': 613}, {'x': 657, 'y': 648}, {'x': 626, 'y': 651}], 'confidence': 0.577}]}, {'text': 'MOTOR', 'boundingPolygon': [{'x': 96, 'y': 686}, {'x': 167, 'y': 680}, {'x': 168, 'y': 698}, {'x': 97, 'y': 704}], 'words': [{'text': 'MOTOR', 'boundingPolygon': [{'x': 96, 'y': 686}, {'x': 163, 'y': 681}, {'x': 163, 'y': 699}, {'x': 98, 'y': 705}], 'confidence': 0.993}]}, {'text': 'TRANS.,AXLE', 'boundingPolygon': [{'x': 97, 'y': 707}, {'x': 224, 'y': 694}, {'x': 226, 'y': 713}, {'x': 98, 'y': 726}], 'words': [{'text': 'TRANS.,AXLE', 'boundingPolygon': [{'x': 98, 'y': 707}, {'x': 221, 'y': 695}, {'x': 222, 'y': 713}, {'x': 99, 'y': 726}], 'confidence': 0.911}]}, {'text': 'FS6R31A HL43', 'boundingPolygon': [{'x': 433, 'y': 669}, {'x': 613, 'y': 651}, {'x': 617, 'y': 685}, {'x': 435, 'y': 703}], 'words': [{'text': 'FS6R31A', 'boundingPolygon': [{'x': 433, 'y': 670}, {'x': 543, 'y': 658}, {'x': 545, 'y': 693}, {'x': 435, 'y': 703}], 'confidence': 0.993}, {'text': 'HL43', 'boundingPolygon': [{'x': 554, 'y': 657}, {'x': 613, 'y': 651}, {'x': 615, 'y': 686}, {'x': 557, 'y': 692}], 'confidence': 0.99}]}, {'text': 'TRANS.,EJE', 'boundingPolygon': [{'x': 97, 'y': 727}, {'x': 208, 'y': 716}, {'x': 210, 'y': 735}, {'x': 98, 'y': 747}], 'words': [{'text': 'TRANS.,EJE', 'boundingPolygon': [{'x': 97, 'y': 727}, {'x': 207, 'y': 717}, {'x': 209, 'y': 735}, {'x': 98, 'y': 747}], 'confidence': 0.986}]}, {'text': 'BUILT', 'boundingPolygon': [{'x': 194, 'y': 743}, {'x': 301, 'y': 727}, {'x': 304, 'y': 770}, {'x': 199, 'y': 784}], 'words': [{'text': 'BUILT', 'boundingPolygon': [{'x': 194, 'y': 742}, {'x': 301, 'y': 727}, {'x': 305, 'y': 770}, {'x': 199, 'y': 785}], 'confidence': 0.994}]}, {'text': '07/2016', 'boundingPolygon': [{'x': 393, 'y': 711}, {'x': 494, 'y': 699}, {'x': 497, 'y': 735}, {'x': 395, 'y': 747}], 'words': [{'text': '07/2016', 'boundingPolygon': [{'x': 393, 'y': 710}, {'x': 493, 'y': 699}, {'x': 498, 'y': 735}, {'x': 397, 'y': 747}], 'confidence': 0.993}]}, {'text': 'PLANT K', 'boundingPolygon': [{'x': 538, 'y': 686}, {'x': 616, 'y': 685}, {'x': 617, 'y': 713}, {'x': 539, 'y': 714}], 'words': [{'text': 'PLANT', 'boundingPolygon': [{'x': 540, 'y': 692}, {'x': 587, 'y': 687}, {'x': 589, 'y': 711}, {'x': 544, 'y': 711}], 'confidence': 0.964}, {'text': 'K', 'boundingPolygon': [{'x': 598, 'y': 687}, {'x': 610, 'y': 686}, {'x': 612, 'y': 714}, {'x': 600, 'y': 712}], 'confidence': 0.766}]}, {'text': 'PLANTA', 'boundingPolygon': [{'x': 540, 'y': 706}, {'x': 604, 'y': 698}, {'x': 607, 'y': 717}, {'x': 542, 'y': 726}], 'words': [{'text': 'PLANTA', 'boundingPolygon': [{'x': 540, 'y': 708}, {'x': 590, 'y': 700}, {'x': 592, 'y': 720}, {'x': 542, 'y': 726}], 'confidence': 0.988}]}, {'text': '3NLCD3387HK802000', 'boundingPolygon': [{'x': 289, 'y': 840}, {'x': 559, 'y': 790}, {'x': 562, 'y': 810}, {'x': 293, 'y': 861}], 'words': [{'text': '3NLCD3387HK802000', 'boundingPolygon': [{'x': 293, 'y': 840}, {'x': 558, 'y': 791}, {'x': 560, 'y': 811}, {'x': 297, 'y': 861}], 'confidence': 0.577}]}]
# lines = [{'text': 'TOYOTA MOTOR THAILAND CO., LTD.', 'boundingPolygon': [{'x': 250, 'y': 150}, {'x': 1222, 'y': 89}, {'x': 1223, 'y': 153}, {'x': 254, 'y': 212}], 'words': [{'text': 'TOYOTA', 'boundingPolygon': [{'x': 255, 'y': 158}, {'x': 445, 'y': 141}, {'x': 448, 'y': 199}, {'x': 258, 'y': 212}], 'confidence': 0.994}, {'text': 'MOTOR', 'boundingPolygon': [{'x': 457, 'y': 140}, {'x': 642, 'y': 126}, {'x': 644, 'y': 186}, {'x': 460, 'y': 198}], 'confidence': 0.997}, {'text': 'THAILAND', 'boundingPolygon': [{'x': 667, 'y': 124}, {'x': 939, 'y': 106}, {'x': 940, 'y': 169}, {'x': 669, 'y': 184}], 'confidence': 0.993}, {'text': 'CO.,', 'boundingPolygon': [{'x': 964, 'y': 104}, {'x': 1087, 'y': 97}, {'x': 1087, 'y': 161}, {'x': 965, 'y': 167}], 'confidence': 0.986}, {'text': 'LTD.', 'boundingPolygon': [{'x': 1099, 'y': 96}, {'x': 1222, 'y': 90}, {'x': 1222, 'y': 155}, {'x': 1099, 'y': 161}], 'confidence': 0.966}]}, {'text': 'MODEL KUN 126L - DTAMYE', 'boundingPolygon': [{'x': 251, 'y': 213}, {'x': 1040, 'y': 162}, {'x': 1042, 'y': 225}, {'x': 254, 'y': 264}], 'words': [{'text': 'MODEL', 'boundingPolygon': [{'x': 251, 'y': 220}, {'x': 380, 'y': 211}, {'x': 382, 'y': 263}, {'x': 253, 'y': 264}], 'confidence': 0.989}, {'text': 'KUN', 'boundingPolygon': [{'x': 393, 'y': 210}, {'x': 512, 'y': 202}, {'x': 513, 'y': 258}, {'x': 394, 'y': 262}], 'confidence': 0.99}, {'text': '126L', 'boundingPolygon': [{'x': 532, 'y': 200}, {'x': 695, 'y': 188}, {'x': 697, 'y': 249}, {'x': 534, 'y': 258}], 'confidence': 0.934}, {'text': '-', 'boundingPolygon': [{'x': 708, 'y': 188}, {'x': 739, 'y': 185}, {'x': 741, 'y': 246}, {'x': 710, 'y': 248}], 'confidence': 0.984}, {'text': 'DTAMYE', 'boundingPolygon': [{'x': 760, 'y': 184}, {'x': 1033, 'y': 163}, {'x': 1034, 'y': 220}, {'x': 761, 'y': 245}], 'confidence': 0.616}]}, {'text': 'ENGINE 1KD -FTV', 'boundingPolygon': [{'x': 267, 'y': 270}, {'x': 710, 'y': 248}, {'x': 713, 'y': 304}, {'x': 269, 'y': 320}], 'words': [{'text': 'ENGINE', 'boundingPolygon': [{'x': 268, 'y': 276}, {'x': 393, 'y': 268}, {'x': 395, 'y': 319}, {'x': 269, 'y': 320}], 'confidence': 0.988}, {'text': '1KD', 'boundingPolygon': [{'x': 422, 'y': 266}, {'x': 524, 'y': 260}, {'x': 526, 'y': 315}, {'x': 424, 'y': 318}], 'confidence': 0.917}, {'text': '-FTV', 'boundingPolygon': [{'x': 541, 'y': 259}, {'x': 698, 'y': 249}, {'x': 702, 'y': 304}, {'x': 544, 'y': 314}], 'confidence': 0.825}]}, {'text': ', 2982.CC', 'boundingPolygon': [{'x': 827, 'y': 237}, {'x': 1202, 'y': 222}, {'x': 1203, 'y': 277}, {'x': 829, 'y': 290}], 'words': [{'text': ',', 'boundingPolygon': [{'x': 831, 'y': 244}, {'x': 859, 'y': 241}, {'x': 860, 'y': 290}, {'x': 833, 'y': 290}], 'confidence': 0.203}, {'text': '2982.CC', 'boundingPolygon': [{'x': 921, 'y': 235}, {'x': 1197, 'y': 222}, {'x': 1198, 'y': 275}, {'x': 922, 'y': 290}], 'confidence': 0.842}]}, {'text': 'FRAME No.', 'boundingPolygon': [{'x': 285, 'y': 329}, {'x': 418, 'y': 325}, {'x': 419, 'y': 366}, {'x': 286, 'y': 370}], 'words': [{'text': 'FRAME', 'boundingPolygon': [{'x': 285, 'y': 331}, {'x': 358, 'y': 329}, {'x': 359, 'y': 368}, {'x': 286, 'y': 370}], 'confidence': 0.989}, {'text': 'No.', 'boundingPolygon': [{'x': 366, 'y': 328}, {'x': 417, 'y': 325}, {'x': 419, 'y': 367}, {'x': 367, 'y': 367}], 'confidence': 0.993}]}, {'text': 'MROHZ8CD8G0502515', 'boundingPolygon': [{'x': 422, 'y': 320}, {'x': 1182, 'y': 282}, {'x': 1183, 'y': 340}, {'x': 425, 'y': 377}], 'words': [{'text': 'MROHZ8CD8G0502515', 'boundingPolygon': [{'x': 423, 'y': 324}, {'x': 1178, 'y': 282}, {'x': 1178, 'y': 341}, {'x': 425, 'y': 377}], 'confidence': 0.663}]}, {'text': 'COLOR', 'boundingPolygon': [{'x': 435, 'y': 382}, {'x': 563, 'y': 379}, {'x': 564, 'y': 415}, {'x': 436, 'y': 419}], 'words': [{'text': 'COLOR', 'boundingPolygon': [{'x': 436, 'y': 382}, {'x': 551, 'y': 379}, {'x': 552, 'y': 415}, {'x': 437, 'y': 420}], 'confidence': 0.997}]}, {'text': 'TRIM GVM(KG)', 'boundingPolygon': [{'x': 603, 'y': 374}, {'x': 946, 'y': 360}, {'x': 947, 'y': 404}, {'x': 604, 'y': 415}], 'words': [{'text': 'TRIM', 'boundingPolygon': [{'x': 617, 'y': 377}, {'x': 733, 'y': 370}, {'x': 733, 'y': 412}, {'x': 618, 'y': 415}], 'confidence': 0.984}, {'text': 'GVM(KG)', 'boundingPolygon': [{'x': 805, 'y': 366}, {'x': 946, 'y': 360}, {'x': 946, 'y': 405}, {'x': 805, 'y': 409}], 'confidence': 0.969}]}, {'text': '070 LA20', 'boundingPolygon': [{'x': 448, 'y': 422}, {'x': 766, 'y': 414}, {'x': 767, 'y': 461}, {'x': 450, 'y': 469}], 'words': [{'text': '070', 'boundingPolygon': [{'x': 449, 'y': 422}, {'x': 553, 'y': 420}, {'x': 555, 'y': 468}, {'x': 451, 'y': 469}], 'confidence': 0.989}, {'text': 'LA20', 'boundingPolygon': [{'x': 606, 'y': 418}, {'x': 756, 'y': 414}, {'x': 759, 'y': 459}, {'x': 608, 'y': 467}], 'confidence': 0.959}]}, {'text': 'TRANS./', 'boundingPolygon': [{'x': 329, 'y': 473}, {'x': 415, 'y': 471}, {'x': 416, 'y': 492}, {'x': 329, 'y': 496}], 'words': [{'text': 'TRANS./', 'boundingPolygon': [{'x': 331, 'y': 474}, {'x': 410, 'y': 471}, {'x': 410, 'y': 493}, {'x': 331, 'y': 496}], 'confidence': 0.865}]}, {'text': 'AXLE', 'boundingPolygon': [{'x': 395, 'y': 487}, {'x': 458, 'y': 484}, {'x': 457, 'y': 509}, {'x': 395, 'y': 510}], 'words': [{'text': 'AXLE', 'boundingPolygon': [{'x': 402, 'y': 486}, {'x': 448, 'y': 484}, {'x': 449, 'y': 508}, {'x': 403, 'y': 510}], 'confidence': 0.864}]}, {'text': 'A75OF CO5B', 'boundingPolygon': [{'x': 457, 'y': 470}, {'x': 854, 'y': 468}, {'x': 854, 'y': 520}, {'x': 457, 'y': 521}], 'words': [{'text': 'A75OF', 'boundingPolygon': [{'x': 466, 'y': 477}, {'x': 640, 'y': 470}, {'x': 641, 'y': 514}, {'x': 467, 'y': 520}], 'confidence': 0.858}, {'text': 'CO5B', 'boundingPolygon': [{'x': 698, 'y': 469}, {'x': 844, 'y': 470}, {'x': 846, 'y': 521}, {'x': 699, 'y': 515}], 'confidence': 0.652}]}, {'text': 'PLANT/', 'boundingPolygon': [{'x': 344, 'y': 522}, {'x': 416, 'y': 521}, {'x': 417, 'y': 543}, {'x': 344, 'y': 544}], 'words': [{'text': 'PLANT/', 'boundingPolygon': [{'x': 346, 'y': 522}, {'x': 416, 'y': 522}, {'x': 416, 'y': 544}, {'x': 346, 'y': 545}], 'confidence': 0.857}]}, {'text': '/BUILT', 'boundingPolygon': [{'x': 394, 'y': 534}, {'x': 460, 'y': 534}, {'x': 460, 'y': 558}, {'x': 393, 'y': 556}], 'words': [{'text': '/BUILT', 'boundingPolygon': [{'x': 399, 'y': 534}, {'x': 459, 'y': 534}, {'x': 458, 'y': 558}, {'x': 399, 'y': 557}], 'confidence': 0.81}]}, {'text': '235 OCT 15', 'boundingPolygon': [{'x': 470, 'y': 519}, {'x': 851, 'y': 512}, {'x': 851, 'y': 556}, {'x': 471, 'y': 562}], 'words': [{'text': '235', 'boundingPolygon': [{'x': 479, 'y': 522}, {'x': 575, 'y': 520}, {'x': 576, 'y': 561}, {'x': 480, 'y': 562}], 'confidence': 0.803}, {'text': 'OCT', 'boundingPolygon': [{'x': 626, 'y': 519}, {'x': 730, 'y': 516}, {'x': 730, 'y': 559}, {'x': 626, 'y': 560}], 'confidence': 0.723}, {'text': '15', 'boundingPolygon': [{'x': 789, 'y': 514}, {'x': 846, 'y': 513}, {'x': 845, 'y': 557}, {'x': 788, 'y': 558}], 'confidence': 0.991}]}, {'text': '02060', 'boundingPolygon': [{'x': 396, 'y': 588}, {'x': 458, 'y': 588}, {'x': 458, 'y': 606}, {'x': 396, 'y': 606}], 'words': [{'text': '02060', 'boundingPolygon': [{'x': 397, 'y': 589}, {'x': 455, 'y': 588}, {'x': 455, 'y': 606}, {'x': 397, 'y': 607}], 'confidence': 0.994}]}]
lines = [{'text': '& OFLA67MYXMDC02159 *', 'boundingPolygon': [{'x': 1230, 'y': 735}, {'x': 237, 'y': 753}, {'x': 236, 'y': 655}, {'x': 1227, 'y': 628}], 'words': [{'text': '&', 'boundingPolygon': [{'x': 1204, 'y': 737}, {'x': 1156, 'y': 738}, {'x': 1153, 'y': 631}, {'x': 1202, 'y': 630}], 'confidence': 0.118}, {'text': 'OFLA67MYXMDC02159', 'boundingPolygon': [{'x': 1126, 'y': 738}, {'x': 298, 'y': 753}, {'x': 296, 'y': 663}, {'x': 1124, 'y': 632}], 'confidence': 0.712}, {'text': '*', 'boundingPolygon': [{'x': 277, 'y': 753}, {'x': 239, 'y': 753}, {'x': 238, 'y': 665}, {'x': 275, 'y': 664}], 'confidence': 0.708}]}, {'text': '22:43:18', 'boundingPolygon': [{'x': 1150, 'y': 281}, {'x': 1041, 'y': 288}, {'x': 1040, 'y': 254}, {'x': 1148, 'y': 249}], 'words': [{'text': '22:43:18', 'boundingPolygon': [{'x': 1149, 'y': 282}, {'x': 1043, 'y': 288}, {'x': 1043, 'y': 255}, {'x': 1149, 'y': 249}], 'confidence': 0.798}]}]

content = {
    'vin'  : {'text': None, 'probability': None},
    'plate': {'text': None, 'probability': None}
}
# start_index = lambda line: min(coord['x'] for coord in line)
# start_index(lines[1]) >= start_index(lines[0])

for line in lines:
    print(repr(line['text']))
    search = re.findall(patterns['vin'], line['text']) 
    if search:
        content['vin']['text'] = search
        word_confidence = [
            (
                word['text'], 
                word['confidence']
            )
            for word in line['words']
            # if len(word['text']) >= 6
        ]
        content['vin']['probability'] = word_confidence#max(word_confidence, key=lambda x: x[1])[1]
        break
    

content

In [ ]:
len('OFLAGTMYXNDC02150')

In [ ]:
start_index = lambda line: min(coord['x'] for coord in line)
start_index(lines[1]) >= start_index(lines[0])
# vmin, vmax = min(x_axis), max(x_axis) 
# print(vmin, vmax)

# get_xaxis = lambda line: [coord['x'] for coord in line]
# x_axis = get_xaxis(lines[1])
# vmin, vmax = min(x_axis), max(x_axis) 
# print(vmin, vmax)


In [ ]:
IOQ_replacement = [
    {'I': '1', 'O': '0', 'Q': '0'},
    {'I': '1', 'O': '9', 'Q': '0'},
]

In [ ]:
import vininfo
OCR_OBSERVED_MISTAKES = {
    # VIN number must only contain alphanumeric symbols except 
    # 'I', 'O', and 'Q'
    'I': ['1'],
    'O': ['0', '9'],
    'Q': ['0', 'O'],

    # Observed errors in OCR
    'A': ['4'],
    'R': ['K'],
    '0': ['O'],
    '5': ['S']
}


# vin = vininfo.Vin('OFLA67MYXMDC02159')
# vin.verify_checksum()

def vin_validation(num):
    try:
        vin = vininfo.Vin(num)
        return vin.verify_checksum()
    except Exception as e:
        return False

num = 'OFLA67MYXMDC02159'
for k, v_list in OCR_OBSERVED_MISTAKES.items():
    for v in v_list:
        print(re.sub(k, v, num), vin_validation(re.sub(k, v, num)))
# [
#     [
#         vin_validation(re.sub(k, v, num))
#         for v in v_list
#     ] 
#     for k, v_list in OCR_OBSERVED_MISTAKES.items()
# ]


In [ ]:
import unicodedata
text = 'XBR. 34G'

def clean_text(text):
    """
    docstring
    """
    text_upper = text.upper()
    text_no_accent = ''.join(                                          
        char for char in unicodedata.normalize('NFD', text_upper)
        if unicodedata.category(char) != 'Mn'
    )
    plate_point_removed = re.sub(
        r'([A-Z]{3})([.]\s)([0-9]{2}[0-9A-Z])', 
        r'\1\3', 
        text_no_accent
    )
    text_alphanumeric = re.sub(r'[^A-Z0-9\s]+', '', plate_point_removed)
    text_no_doublespaces = re.sub(r'\s+', ' ', text_alphanumeric)

    return text_no_doublespaces

            
clean_text(text)
    
    

In [ ]:
def detect_plate(self):
    """
    Detects and extracts license plates from an image using the Azure
    AI Custom Vision API for object detection.
    """

    def crop_image(image, bounding_box):
        """
        docstring
        """
        # Extract region defined by the bounding box
        left   = bounding_box['left']   * image.width
        top    = bounding_box['top']    * image.height
        width  = bounding_box['width']  * image.width
        height = bounding_box['height'] * image.height

        # Crop the image
        img_cropped = image.crop((
            left, 
            top, 
            left + width, 
            top + height
        ))

        img_cropped_location = \
            f'work-zone/Resultados/{os.path.basename('TEMP.png')}'

        # # Plot the cropped image
        # plt.imshow(img_cropped)
        # plt.axis('off')
        # plt.savefig(
        #     f'work-zone/Resultados/TEST.png', 
        #     bbox_inches='tight', 
        #     pad_inches=0
        # )
        # plt.show()

        return img_cropped_location


    self.content['applied_fuctions'].append('detect_plate')

    # Call Azure AI Custom vision API (object-detection) -----------
    img_data = BytesIO(self.img_binary)  # original_image = Image.open(self.img_binary)
    original_image = Image.open(img_data)

    response = requests.request(
        method='POST', 
        url=ENDPOINT_PLATE_DETECTION, 
        headers=HEADERS_CUSTOM_VISION, 
        data=self.img_binary
    )
    # --------------------------------------------------------------

    predictions = response.json()['predictions']

    if predictions and (predictions[0]['probability'] 
                        >= THRESHOLD_PLATE_DETECTION
                        ):
        bounding_box = predictions[0]['boundingBox']
        self.content['plate']['probability']  = \
            predictions[0]['probability']

        # Cropped Image --------------------------------------------
        img_cropped_location = crop_image(
            original_image, 
            bounding_box
        )
        img_cropped = VehicleImage(
            img_binary=img_cropped_location
        )
        img_cropped.extract_text(content_type='plate')
        self.content['applied_fuctions'].append('extract_text:cropped')
        img_cropped.show()
        # ----------------------------------------------------------

        self.content['plate']['text'] = \
            img_cropped.content['plate']['text']

    else:  # NO PLATE DETECTED
        self.content['status'] = 'REVIEW'
        img_cropped = None

In [ ]:
num = 'OFLA67MYXMDC02159'

IOQ_replacement = {
    'I': ['1'],
    'O': ['0', '9'],
    'Q': ['0', 'O'],
}

In [ ]:
from itertools import product

generate_variations = lambda char: [char] if char not in 'IOQ' else [char] + IOQ_replacement[char]
possibilities = [generate_variations(n) for n in num]
all_combinations = product(*possibilities)

for combination in all_combinations:
    posibility = ''.join(combination)
    vin_validated = vin_validation(posibility)
    if vin_validated:
        print(posibility, vin_validated)
        break



In [ ]:

# Extended dictionary of common OCR confusions
OCR_CONFUSIONS = {
    # 'I': ['1'],
    # 'O': ['0', '9'],
    # 'Q': ['0'],

    # # Observed errors in OCR
    # 'A': ['4'],
    # 'R': ['K'],
    # '0': ['O'],
    # '5': ['S'],
    'I': ['1'],
    'O': ['0', '9'],
    'Q': ['0', 'O'],
}

def generate_variations(text):
    # Generate a list of possible characters for each position
    possibilities = []
    for char in text:
        if char in OCR_CONFUSIONS:
            possibilities.append([char] + OCR_CONFUSIONS[char])
        else:
            possibilities.append([char])
    
    # Use itertools.product to generate all combinations
    all_combinations = list(product(*possibilities))
    
    # Join each combination tuple into a string
    variations = [''.join(combination) for combination in all_combinations]
    print(variations)
    
    return variations

# Example usage
vin = 'OFLA67MYXMDC02159'
variations = generate_variations(vin)

# Print the first 10 variations as an example
print(f'Total variations: {len(variations)}')
print('First 10 variations:')
for variation in variations[:10]:
    print(variation, vin_validation(variation))

In [ ]:
lines = [{'text': 'NISSAN', 'boundingPolygon': [{'x': 361, 'y': 427}, {'x': 578, 'y': 428}, {'x': 578, 'y': 476}, {'x': 360, 'y': 475}], 'words': [{'text': 'NISSAN', 'boundingPolygon': [{'x': 363, 'y': 427}, {'x': 571, 'y': 429}, {'x': 570, 'y': 477}, {'x': 361, 'y': 476}], 'confidence': 0.994}]}, {'text': 'TYPE', 'boundingPolygon': [{'x': 302, 'y': 479}, {'x': 335, 'y': 479}, {'x': 335, 'y': 498}, {'x': 302, 'y': 499}], 'words': [{'text': 'TYPE', 'boundingPolygon': [{'x': 302, 'y': 479}, {'x': 334, 'y': 479}, {'x': 334, 'y': 499}, {'x': 302, 'y': 499}], 'confidence': 0.979}]}, {'text': 'TIPO', 'boundingPolygon': [{'x': 300, 'y': 501}, {'x': 332, 'y': 500}, {'x': 333, 'y': 520}, {'x': 301, 'y': 520}], 'words': [{'text': 'TIPO', 'boundingPolygon': [{'x': 301, 'y': 500}, {'x': 331, 'y': 500}, {'x': 332, 'y': 520}, {'x': 301, 'y': 520}], 'confidence': 0.967}]}, {'text': 'NO. DE CHASSIS', 'boundingPolygon': [{'x': 98, 'y': 552}, {'x': 262, 'y': 545}, {'x': 263, 'y': 566}, {'x': 99, 'y': 574}], 'words': [{'text': 'NO.', 'boundingPolygon': [{'x': 98, 'y': 553}, {'x': 136, 'y': 551}, {'x': 137, 'y': 572}, {'x': 99, 'y': 574}], 'confidence': 0.996}, {'text': 'DE', 'boundingPolygon': [{'x': 141, 'y': 551}, {'x': 167, 'y': 550}, {'x': 167, 'y': 571}, {'x': 141, 'y': 572}], 'confidence': 0.989}, {'text': 'CHASSIS', 'boundingPolygon': [{'x': 177, 'y': 549}, {'x': 260, 'y': 546}, {'x': 260, 'y': 567}, {'x': 177, 'y': 570}], 'confidence': 0.993}]}, {'text': '3N6CD33B7HK802000', 'boundingPolygon': [{'x': 316, 'y': 522}, {'x': 627, 'y': 511}, {'x': 628, 'y': 549}, {'x': 317, 'y': 561}], 'words': [{'text': '3N6CD33B7HK802000', 'boundingPolygon': [{'x': 319, 'y': 523}, {'x': 626, 'y': 511}, {'x': 626, 'y': 550}, {'x': 320, 'y': 562}], 'confidence': 0.993}]}, {'text': 'D23', 'boundingPolygon': [{'x': 476, 'y': 477}, {'x': 528, 'y': 477}, {'x': 528, 'y': 515}, {'x': 476, 'y': 515}], 'words': [{'text': 'D23', 'boundingPolygon': [{'x': 476, 'y': 477}, {'x': 527, 'y': 477}, {'x': 527, 'y': 515}, {'x': 476, 'y': 515}], 'confidence': 0.998}]}, {'text': 'CHASSIS NO.', 'boundingPolygon': [{'x': 99, 'y': 528}, {'x': 230, 'y': 525}, {'x': 231, 'y': 545}, {'x': 99, 'y': 550}], 'words': [{'text': 'CHASSIS', 'boundingPolygon': [{'x': 99, 'y': 529}, {'x': 186, 'y': 527}, {'x': 186, 'y': 547}, {'x': 99, 'y': 550}], 'confidence': 0.994}, {'text': 'NO.', 'boundingPolygon': [{'x': 195, 'y': 527}, {'x': 230, 'y': 526}, {'x': 230, 'y': 546}, {'x': 195, 'y': 547}], 'confidence': 0.994}]}, {'text': 'MODEL', 'boundingPolygon': [{'x': 98, 'y': 574}, {'x': 170, 'y': 573}, {'x': 170, 'y': 593}, {'x': 99, 'y': 596}], 'words': [{'text': 'MODEL', 'boundingPolygon': [{'x': 98, 'y': 575}, {'x': 167, 'y': 573}, {'x': 168, 'y': 593}, {'x': 99, 'y': 597}], 'confidence': 0.996}]}, {'text': 'CVL4LWYD23IYP-', 'boundingPolygon': [{'x': 293, 'y': 563}, {'x': 521, 'y': 552}, {'x': 523, 'y': 591}, {'x': 295, 'y': 603}], 'words': [{'text': 'CVL4LWYD23IYP-', 'boundingPolygon': [{'x': 294, 'y': 564}, {'x': 519, 'y': 553}, {'x': 522, 'y': 590}, {'x': 295, 'y': 604}], 'confidence': 0.879}]}, {'text': 'MODELO', 'boundingPolygon': [{'x': 98, 'y': 598}, {'x': 182, 'y': 593}, {'x': 184, 'y': 612}, {'x': 98, 'y': 618}], 'words': [{'text': 'MODELO', 'boundingPolygon': [{'x': 98, 'y': 599}, {'x': 180, 'y': 594}, {'x': 180, 'y': 613}, {'x': 98, 'y': 619}], 'confidence': 0.996}]}, {'text': 'COLOR,TRIM', 'boundingPolygon': [{'x': 97, 'y': 620}, {'x': 224, 'y': 612}, {'x': 225, 'y': 632}, {'x': 98, 'y': 641}], 'words': [{'text': 'COLOR,TRIM', 'boundingPolygon': [{'x': 98, 'y': 621}, {'x': 213, 'y': 613}, {'x': 214, 'y': 632}, {'x': 98, 'y': 641}], 'confidence': 0.962}]}, {'text': 'KH3 G', 'boundingPolygon': [{'x': 461, 'y': 592}, {'x': 535, 'y': 588}, {'x': 537, 'y': 622}, {'x': 462, 'y': 629}], 'words': [{'text': 'KH3', 'boundingPolygon': [{'x': 461, 'y': 592}, {'x': 514, 'y': 588}, {'x': 517, 'y': 624}, {'x': 462, 'y': 628}], 'confidence': 0.993}, {'text': 'G', 'boundingPolygon': [{'x': 521, 'y': 588}, {'x': 533, 'y': 588}, {'x': 536, 'y': 623}, {'x': 524, 'y': 623}], 'confidence': 0.993}]}, {'text': 'COLOR, GUARNICION', 'boundingPolygon': [{'x': 97, 'y': 642}, {'x': 286, 'y': 628}, {'x': 287, 'y': 648}, {'x': 98, 'y': 663}], 'words': [{'text': 'COLOR,', 'boundingPolygon': [{'x': 98, 'y': 642}, {'x': 167, 'y': 637}, {'x': 168, 'y': 658}, {'x': 98, 'y': 663}], 'confidence': 0.993}, {'text': 'GUARNICION', 'boundingPolygon': [{'x': 171, 'y': 637}, {'x': 285, 'y': 628}, {'x': 286, 'y': 648}, {'x': 172, 'y': 658}], 'confidence': 0.993}]}, {'text': 'ENGINE', 'boundingPolygon': [{'x': 98, 'y': 664}, {'x': 171, 'y': 659}, {'x': 172, 'y': 678}, {'x': 99, 'y': 683}], 'words': [{'text': 'ENGINE', 'boundingPolygon': [{'x': 98, 'y': 664}, {'x': 170, 'y': 660}, {'x': 170, 'y': 679}, {'x': 99, 'y': 684}], 'confidence': 0.994}]}, {'text': 'YD25DDTI 2488 cc', 'boundingPolygon': [{'x': 429, 'y': 633}, {'x': 656, 'y': 612}, {'x': 659, 'y': 647}, {'x': 432, 'y': 668}], 'words': [{'text': 'YD25DDTI', 'boundingPolygon': [{'x': 430, 'y': 634}, {'x': 559, 'y': 622}, {'x': 562, 'y': 658}, {'x': 433, 'y': 668}], 'confidence': 0.943}, {'text': '2488', 'boundingPolygon': [{'x': 566, 'y': 622}, {'x': 616, 'y': 617}, {'x': 619, 'y': 652}, {'x': 569, 'y': 657}], 'confidence': 0.993}, {'text': 'cc', 'boundingPolygon': [{'x': 623, 'y': 616}, {'x': 654, 'y': 613}, {'x': 657, 'y': 648}, {'x': 626, 'y': 651}], 'confidence': 0.577}]}, {'text': 'MOTOR', 'boundingPolygon': [{'x': 96, 'y': 686}, {'x': 167, 'y': 680}, {'x': 168, 'y': 698}, {'x': 97, 'y': 704}], 'words': [{'text': 'MOTOR', 'boundingPolygon': [{'x': 96, 'y': 686}, {'x': 163, 'y': 681}, {'x': 163, 'y': 699}, {'x': 98, 'y': 705}], 'confidence': 0.993}]}, {'text': 'TRANS.,AXLE', 'boundingPolygon': [{'x': 97, 'y': 707}, {'x': 224, 'y': 694}, {'x': 226, 'y': 713}, {'x': 98, 'y': 726}], 'words': [{'text': 'TRANS.,AXLE', 'boundingPolygon': [{'x': 98, 'y': 707}, {'x': 221, 'y': 695}, {'x': 222, 'y': 713}, {'x': 99, 'y': 726}], 'confidence': 0.911}]}, {'text': 'FS6R31A HL43', 'boundingPolygon': [{'x': 433, 'y': 669}, {'x': 613, 'y': 651}, {'x': 617, 'y': 685}, {'x': 435, 'y': 703}], 'words': [{'text': 'FS6R31A', 'boundingPolygon': [{'x': 433, 'y': 670}, {'x': 543, 'y': 658}, {'x': 545, 'y': 693}, {'x': 435, 'y': 703}], 'confidence': 0.993}, {'text': 'HL43', 'boundingPolygon': [{'x': 554, 'y': 657}, {'x': 613, 'y': 651}, {'x': 615, 'y': 686}, {'x': 557, 'y': 692}], 'confidence': 0.99}]}, {'text': 'TRANS.,EJE', 'boundingPolygon': [{'x': 97, 'y': 727}, {'x': 208, 'y': 716}, {'x': 210, 'y': 735}, {'x': 98, 'y': 747}], 'words': [{'text': 'TRANS.,EJE', 'boundingPolygon': [{'x': 97, 'y': 727}, {'x': 207, 'y': 717}, {'x': 209, 'y': 735}, {'x': 98, 'y': 747}], 'confidence': 0.986}]}, {'text': 'BUILT', 'boundingPolygon': [{'x': 194, 'y': 743}, {'x': 301, 'y': 727}, {'x': 304, 'y': 770}, {'x': 199, 'y': 784}], 'words': [{'text': 'BUILT', 'boundingPolygon': [{'x': 194, 'y': 742}, {'x': 301, 'y': 727}, {'x': 305, 'y': 770}, {'x': 199, 'y': 785}], 'confidence': 0.994}]}, {'text': '07/2016', 'boundingPolygon': [{'x': 393, 'y': 711}, {'x': 494, 'y': 699}, {'x': 497, 'y': 735}, {'x': 395, 'y': 747}], 'words': [{'text': '07/2016', 'boundingPolygon': [{'x': 393, 'y': 710}, {'x': 493, 'y': 699}, {'x': 498, 'y': 735}, {'x': 397, 'y': 747}], 'confidence': 0.993}]}, {'text': 'PLANT K', 'boundingPolygon': [{'x': 538, 'y': 686}, {'x': 616, 'y': 685}, {'x': 617, 'y': 713}, {'x': 539, 'y': 714}], 'words': [{'text': 'PLANT', 'boundingPolygon': [{'x': 540, 'y': 692}, {'x': 587, 'y': 687}, {'x': 589, 'y': 711}, {'x': 544, 'y': 711}], 'confidence': 0.964}, {'text': 'K', 'boundingPolygon': [{'x': 598, 'y': 687}, {'x': 610, 'y': 686}, {'x': 612, 'y': 714}, {'x': 600, 'y': 712}], 'confidence': 0.766}]}, {'text': 'PLANTA', 'boundingPolygon': [{'x': 540, 'y': 706}, {'x': 604, 'y': 698}, {'x': 607, 'y': 717}, {'x': 542, 'y': 726}], 'words': [{'text': 'PLANTA', 'boundingPolygon': [{'x': 540, 'y': 708}, {'x': 590, 'y': 700}, {'x': 592, 'y': 720}, {'x': 542, 'y': 726}], 'confidence': 0.988}]}, {'text': '3NLCD3387HK802000', 'boundingPolygon': [{'x': 289, 'y': 840}, {'x': 559, 'y': 790}, {'x': 562, 'y': 810}, {'x': 293, 'y': 861}], 'words': [{'text': '3NLCD3387HK802000', 'boundingPolygon': [{'x': 293, 'y': 840}, {'x': 558, 'y': 791}, {'x': 560, 'y': 811}, {'x': 297, 'y': 861}], 'confidence': 0.577}]}]
for line in lines:
    print(repr(line['text']))
    for word in line['words']:
        search = re.search(r'\b(?=.*[0-9])(?=.*[A-Z])[0-9A-Z]{17}\b', word['text'])
        print('\t', repr(word['text']), word['confidence'], bool(search))
        print()